<center>

# **Đồ án cuối kì**

# **Môn Lập trình cho khoa học dữ liệu**

## **Chủ đề: Các vụ án mạng ở Mỹ từ năm 2016 đến năm 2020**

</center>

<center>

# **Mục lục**

</center>

# **Các thư viện sử dụng**

In [ ]:
# Libraries and Modules used
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #visualization
sns.set_style('whitegrid')

---

# **Mô tả tập dữ liệu**

- Tập dữ liệu này được thu thập từ [**Murder Accountability Project**](https://www.murderdata.org/p/data-docs.html), do Thomas Hargrove sáng lập và công bố. Nhìn chung, tập dữ liệu mô tả về các vụ án giết người được xảy ra ở khắp mọi bang trên nước Mỹ từ năm 1976 đến năm 2020.

- Tập dữ liệu này là một tập dữ liệu mở được công bố dưới bản quyền [***Creative Commons CC-BY license***](https://creativecommons.org/licenses/by/4.0/), cho phép sử dụng không hạn chế dưới mọi hình thức, miễn là người dùng giữ nguyên tập dữ liệu gốc.

- Tập dữ liệu bao gồm những vụ án mạng được tổng hợp từ báo cáo của FBI và dữ liệu của Freedom of Information Act về các vụ án mạng không được báo cáo cho Justice Department.

- Do tập dữ liệu có kích cỡ quá lớn (khoảng 290MB) nên nhóm chúng em chỉ lọc lấy những vụ án từ năm 2016 đến năm 2020.

---

# **Tiền xử lý dữ liệu**

## Đọc dữ liệu vào Dataframe

In [ ]:
homi_df = pd.read_csv('homicide.csv')
homi_df

## Số dòng và số cột của tập dữ liệu

In [ ]:
row = homi_df.shape[0]
col = homi_df.shape[1]
print('Số dòng:', row)
print('Số cột', col)

## Ý nghĩa của mỗi dòng

Mỗi dòng là thông tin của 1 vụ án bao gồm thời gian, địa điểm và thông tin liên quan đến nạn nhân và hung thủ.

## Kiểm tra các dòng bị trùng lắp

- Thực hiện xóa các dòng bị trùng lắp do trong đó có nhiều nạn nhân liên quan tới cùng 1 vụ án.

In [ ]:
# homi_df = homi_df['ID'].ddrop_duplicates() 

In [ ]:
# duplicated = homi_df.duplicated()
# print('Số dòng bị trùng:', len(duplicated[duplicated==True]))

## Ý nghĩa của mỗi cột

In [ ]:
des_df = pd.read_csv('description.csv')
pd.set_option('display.max_colwidth', None)
des_df

## Kiểm tra kiểu dữ liệu của các cột và điều chỉnh cho phù hợp

- Kiểm tra kiểu dữ liệu của từng cột

In [ ]:
homi_df.dtypes

---

In [ ]:
homi_df['StateName'].unique()

In [ ]:
dtype = homi_df['StateName'].apply(lambda i: type(i))
dtype.unique()

In [ ]:
len(homi_df[homi_df['StateName'].isna()])

- Ta thấy cột `StateName` chứa dữ liệu hỗn hợp với dữ liệu `nan` là float và tên các bang là string. Tuy nhiên có đến 69378 dòng là nan. Hơn nữa, tên bang cũng đã được lưu trong cột dữ liệu `State`. Cho nên, ta có thể xóa cột dữ liệu này (ở bước ***Xử lý các dòng bị thiếu dữ liệu***).

---

In [ ]:
np.sort(homi_df['VicAge'].unique())

- Ta thấy có những dòng mà tuổi của nạn nhân là 999. Ở đây, do kiểu dữ liệu của cột VicAge là số nguyên cho nên tác giả dùng 1 con số đặc biệt để biểu thị cho giá trị không biết của tuổi nạn nhân, cụ thể là 999. Việc này có thể xem bình thường cho nên không cần thay đổi gì cả.

---

In [ ]:
np.sort(homi_df['OffAge'].unique())

- Đối với cột OffAge thì lý giải cũng tương tự như đối với cột VicAge.

---

In [ ]:
homi_df['Subcircum'].unique()

In [ ]:
dtype = homi_df['Subcircum'].apply(lambda i: type(i))
dtype.unique()

- Ta thấy cột `Subcircum` cũng chứa dữ liệu hỗn hợp như cột StateName. Tuy nhiên, các dòng chứa dữ liệu khác `nan` có vai trò làm rõ cho hoàn cảnh gây án. Do đó, chúng ta sẽ không bỏ cột này đi, và thay các giá trị `nan` bằng `Not specified`.

In [ ]:
len(homi_df[homi_df['Subcircum'].isna()])

In [ ]:
homi_df['Subcircum'].replace(np.nan, 'Not specified', inplace=True)

In [ ]:
len(homi_df[homi_df['Subcircum'] == 'Not specified'])

- Kiểm tra lại kiểu dữ liệu

In [ ]:
new_dtype = homi_df['Subcircum'].apply(lambda i: type(i))
new_dtype.unique()

---

In [ ]:
homi_df['FileDate'].unique()

In [ ]:
len(homi_df[homi_df['FileDate'].isna()])

- Ở đây, dữ liệu (khác `nan`) của mỗi dòng là dãy số tượng trưng cho ngày, tháng, năm của thời điểm thực hiện báo cáo về vụ án. Những dòng này đúng ra phải nên được chuyển về dang datetime. Tuy nhiên do tính liên quan đến vấn đề không được chặt chẽ lắm (thời điểm thực hiện báo cáo có thể không trùng với thời điểm gây án, và có thể chỉ là thời điểm được ước tính), nên ta có thể bỏ cột này đi (ở bước ***Xử lý các dòng bị thiếu dữ liệu***).

---

- Kiểm tra kiểu dữ liệu của tất cả các cột 1 lần nữa

In [ ]:
homi_df.dtypes

- Kiểm tra lại toàn bộ tập dữ liệu

In [ ]:
homi_df

---

In [ ]:
homi_df['FileDate'].nunique()

## Tìm sự phân phối của các cột dữ liệu kiểu số

In [ ]:
col_num = ['Year','Incident','VicAge','OffAge','VicCount','OffCount']

num_df = homi_df[col_num]

mising_percentage = (num_df.isna().sum() / row) * 100       # tỉ lệ giá trị bị thiếu
min_values = num_df.min()                                   # các giá trị nhỏ nhất
max_values = num_df.max()                                   # các giá trị lớn nhất

num_cols_distribution = pd.DataFrame(data = np.array((mising_percentage, min_values, max_values)),
                                                      columns = num_df.columns,
                                                      index = ['mising percentage', 'min', 'max'])
num_cols_distribution

## Tìm sự phân phối của các cột dữ liệu kiểu phân loại

In [ ]:
cat_df = homi_df.copy().drop(columns=col_num)

mising_percentage = (cat_df.isna().sum() / row) * 100           # tỉ lệ giá trị bị thiếu
num_diff_vals = cat_df.apply(lambda col: col.nunique())         # số giá trị khác nhau
diff_vals = cat_df.apply(lambda col: col.dropna().unique())     # các giá trị khác nhau

cat_cols_distribution = pd.DataFrame(data = np.array((mising_percentage, num_diff_vals, diff_vals)),
                                     columns = cat_df.columns,
                                     index = ['mising percentage', 'num_diff_vals', 'diff_vals'])
cat_cols_distribution

## Xử lý các dòng bị thiếu dữ liệu

- Xóa các cột `StateName` và `FileDate` như đã đề cập ở bước ***Kiểm tra dữ liệu của các cột và điều chỉnh cho phù hợp***.

In [ ]:
homi_df = homi_df.drop(columns = ['StateName', 'FileDate'], axis = 1)

In [ ]:
homi_df

---

# **Phân tích dữ liệu**

## **Câu hỏi số 1**

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 2**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 3**

### **3 bang nào có các vụ án có thủ phạm là người trong gia đình nhiều nhất.**

- Từ việc trả lời câu hỏi trên, ta có thể đưa ra những phương pháp để bảo vệ khỏi những chuyện đáng tiếc không nên xảy ra, nâng cao ý thức cũng như trách nhiệm của mỗi người trong gia đình.

### Tiền xử lý

- Để có thể lấy được những vụ án có thủ phạm là người trong gia đình thì ta sẽ lấy những cột có liên quan như: 
    - ID: Để có thể chọn lọc những vụ án.
    - State: Để tìm ra những ban có liên quan.
    - Year: Để xem chiều hướng của các vụ án đó.
    - Homicide: Để chọn được những vụ án có liên quan đến câu hỏi.
    - Relationship: Để tìm ra mối quan hệ ruột thịt giữa nạn nhân và hung thủ.   


In [ ]:
df = homi_df[['ID', 'State', 'Year', 'Homicide', 'Relationship']]
df

- Trước tiên ta xóa những dòng trùng lắp ID của các vụ báo cáo vì: Trong data của chúng ta, báo cáo sẽ bao gồm thông tin của nạn nhân và hung thủ. Mà có thể bị trùng ID do là nạn nhân khác trong cùng một vụ án.
    

In [ ]:
df = df.drop_duplicates(subset='ID', keep="first", inplace= False).reset_index(drop = True)
df

### Phân tích

- Bây giờ ta sẽ xem xét mối quan hệ của từng vụ án

In [ ]:
relationship = df['Relationship'].unique()
relationship

- Từ đó, ta có thể chọn lọc được những vụ án mà hung thủ có mối quan hệ trong gia đình với nạn nhân. Ví dụ như là: Husband, Wife, Sister, In-law, Stepfather, Stepson, Daughter, Brother, Stepmother, Common-law wife, Common-law husband, Stepdaughter. Nhưng ở đây ta sẽ không chọn những mối quan hệ như `Ex-wife`, `Ex-husband` vì đã không còn trong mối quan hệ gia đình.

In [ ]:
family_rel = ['Husband', 'Wife', 'Sister', 'In-law', 'Stepfather', 'Stepson', 'Daughter', 'Brother', 'Stepmother', 'Common-law wife', 'Common-law husband', 'Stepdaughter']
family_df = df[df['Relationship'].isin(family_rel)]
family_df

Tiếp theo, ta sẽ thống kê các vụ án cũng như là đếm từng vụ án theo mối quan hệ theo từng bang.

In [ ]:
count = pd.DataFrame(family_df.groupby(['State', 'Relationship']).size().rename('Count')).reset_index()
count

Sau khi thống kê các vụ án, ta sẽ thực hiện bước tính tổng các vụ án và sắp xếp theo chiều hướng giảm dần để có thể nhìn thấy được bang nào có nhiều vụ án nhất. 

In [ ]:
count_state = count.groupby('State').apply(lambda x: x['Count'].sum()).rename('Sum').reset_index()
count_state = count_state.sort_values(by=['Sum'], ascending = False)
count_state 

Ở phần đặt câu hỏi này, ta sẽ lấy 3 bang có nhiều vụ án có hung thủ là người trong gia đình để ta có thể dễ nhìn hơn. Khi có được 3 bang, ta có thể dễ dàng tìm thấy 5 mối quan hệ có liên quan đến án mạng nhiều nhất.

In [ ]:
top_3_state = count_state['State'][:3].values
top_3_state_incident = count[count['State'].isin(top_3_state)]
top_3_rel_incident = top_3_state_incident.groupby('State').apply(lambda x: x.nlargest(5, 'Count')).reset_index(drop=True)
top_3_rel_incident 

### Trực quan hóa

- Còn đây là top 3 bang có số vụ án liên quan đến gia đình nhiều nhất.

In [ ]:
top_3_state = count_state[:3]

# plt.figure(figsize = (10, 5))

plt.title("Top 3 States with most incidents relating to family relationship")
plt.ylabel("Number of incidents")
plt.xlabel("State")
plt.bar(top_3_state['State'], top_3_state['Sum'], width=0.5)
plt.show()

- Từ việc phân tích trên, ta có thể dễ dàng vẽ được biểu đồ về 5 mối quan hệ của  mỗi bang có liên quan đến án mạng nhất.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (20, 10))

texas_df = top_3_rel_incident[top_3_rel_incident['State'] == 'Texas']
axs[0].set_title('TEXAS')
axs[0].set(xlabel = 'Relationship', ylabel = 'Number of incident')
axs[0].bar(texas_df['Relationship'], texas_df['Count'])
axs[0].set_xticks(texas_df['Relationship'])
axs[0].set_xticklabels(texas_df['Relationship'], rotation = 40);

california_df = top_3_rel_incident[top_3_rel_incident['State'] == 'California']
axs[1].set_title('CALIFORNIA')
axs[1].set(xlabel = 'Relationship', ylabel = 'Number of incident')
axs[1].bar(california_df['Relationship'], california_df['Count'])
axs[1].set_xticks(california_df['Relationship'])
axs[1].set_xticklabels(california_df['Relationship'], rotation = 40);

florida_df = top_3_rel_incident[top_3_rel_incident['State'] == 'Florida']
axs[2].set_title('FLORIDA')
axs[2].set(xlabel = 'Relationship', ylabel = 'Number of incident')
axs[2].bar(florida_df['Relationship'], florida_df['Count'])
axs[2].set_xticks(florida_df['Relationship'])
axs[2].set_xticklabels(florida_df['Relationship'], rotation = 40);


### Trả lời câu hỏi

- Quan sát biểu đồ top 5 mối quan hệ có số vụ án nhiều nhất, ta thấy số vụ án có vợ là nạn nhân cao hơn tương đối nhiều so với các quan hệ còn lại. Do đó các quan chức năng cần quan tâm hơn và chú trọng các quyền bảo vệ phụ nữ nhiều hơn. Bên cạnh đó, họ cần đặc biệt chú ý về vấn đề bạo lực gia đình của chồng đối với vợ, bởi vì đây rất có thể sẽ là mầm mống dẫn tới các vụ án mạng có liên quan. Sâu xa hơn nữa, hệ thống giáo dục cần đề cao việc giáo dục trong gia đình để không xảy ra những sự việc đáng tiếc. Một gia đình hạnh phúc mới có thể tạo nên một xã hội tốt đẹp.

---

## **Câu hỏi số 4**

### **So sánh số vụ án của các chủng tộc phạm tội theo độ tuổi**

- Từ việc trả lời câu hỏi này, ta có thể so sánh được trình độ dân trí giữa các chủng tộc theo từng nhóm tuổi. Từ đó, đưa ra những chính sách giáo dục hợp lí để cải thiện xã hội ở Mỹ.

### Tiền xử lý

- Để có thể trả lời câu hỏi này, chúng ta chỉ cần lấy 1 vài cột như:
    - ID: Để chọn lọc vụ án
    - OffRace: Để có thể so sánh các nhóm chủng tộc của hung thủ
    - OffAge: Đê chia nhóm tuổi của các hung thủ

In [ ]:
col_name = ['ID', 'OffRace', 'OffAge']
df = homi_df[col_name].copy()
df

- Tiếp theo ta sẽ loại bỏ những vụ án có trùng ID vì: Những vụ án đó có thể là do có nhiều nạn nhân trong cùng 1 một vụ án nên nó sẽ tạo ra nhiều báo cáo có cùng ID.

In [ ]:
df = df.drop_duplicates(subset='ID', keep="first", inplace= False).reset_index(drop = True)
df

- Ta sẽ loại bỏ những dòng có độ tuổi không phù hợp như: 999. Đó có thể là do tác giả định nghĩa không xác điịnh độ tuổi thành 999 để có thể phân biệt rõ với tuổi khác.

In [ ]:
df = df[df['OffAge'] != 999]
df

### Phân tích

- Trước tiên chúng ta sẽ tạo 1 hàm để chia độ tuổi cho phù hợp. Ở đây ta sẽ chia làm 3 nhóm tuổi: [0, 17],[18, 40], (40, 99). Sau đó ta sẽ tạo thêm cột GroupAge.

In [ ]:
bins = [0, 17, 40,99]
labels = ['0-17', '18-40', '40-99']
df['GroupAge'] = pd.cut(df['OffAge'], bins=bins, labels = labels,include_lowest = True)
df

- Lúc này, chúng ta bỏ 2 cột không cần thiết nữa đó là ID và OffAge.

In [ ]:
df = df.drop(['ID','OffAge'], axis=1)
df

- Cuối cùng ta sẽ tính tổng số vụ án theo độ tuổi quy định ban đầu cho mỗi chủng tộc.

In [ ]:
count = pd.DataFrame(df.groupby(['OffRace', 'GroupAge']).size().rename('Count')).reset_index()
count

### Trực quan hóa

- Từ dữ liệu trên, ta có thể vẽ biểu đồ như sau:

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (20, 15),  sharey=True)

Race_1 = count[count['OffRace'] == 'Black']
axs[0,0].set_title('Black')
axs[0,0].bar([0, 0.5, 1], Race_1['Count'], width = 0.4)
axs[0,0].set_xticks([0, 0.5, 1], Race_1['GroupAge'])

Race_2 = count[count['OffRace'] == 'American Indian or Alaskan Native']
axs[0,1].set_title('American Indian or Alaskan Native')
axs[0,1].bar([0, 0.5, 1], Race_2['Count'], width = 0.4)
axs[0,1].set_xticks([0, 0.5, 1], Race_2['GroupAge'])

Race_3 = count[count['OffRace'] == 'Asian']
axs[0,2].set_title('Asian')
axs[0,2].bar([0, 0.5, 1], Race_3['Count'], width = 0.4)
axs[0,2].set_xticks([0, 0.5, 1], Race_3['GroupAge'])

Race_4 = count[count['OffRace'] == 'White']
axs[1,0].set_title('White')
axs[1,0].bar([0, 0.5, 1], Race_4['Count'], width = 0.4)
axs[1,0].set_xticks([0, 0.5, 1], Race_4['GroupAge'])

Race_5 = count[count['OffRace'] == 'Native Hawaiian or Pacific Islander']
axs[1,1].set_title('Native Hawaiian or Pacific Islander')
axs[1,1].bar([0, 0.5, 1], Race_5['Count'], width = 0.4)
axs[1,1].set_xticks([0, 0.5, 1], Race_5['GroupAge'])

Race_6 = count[count['OffRace'] == 'Unknown']
axs[1,2].set_title('Unknown')
axs[1,2].bar([0, 0.5, 1], Race_6['Count'], width = 0.4)
axs[1,2].set_xticks([0, 0.5, 1], Race_6['GroupAge']);


### Trả lời câu hỏi

- Nhìn sơ bộ về biểu đồ, ta thấy hung thủ của các vụ án thường liên quan đến 2 chủng tộc chính là người da trắng và người da đen, tập trung chủ yếu ở nhóm tuổi từ 18-40. Tuy số hung thủ là người da đen nhiều hơn số hung thủ là người da trắng, số vụ án gây ra của 2 chủng tộc này vẫn ở mức khá cao. Từ đó, có thể suy ra 1 phần nào đó của các vụ án này có liên quan đến nạn phân biệt chủng tộc. Vì vậy, các bang chức trách cần tuyên truyền và nâng cao nhận thức cho người dân, giáo dục cho các thế hệ trẻ ý thức để chống lại nạn phân biệt chủng tộc nhằm hạn chế số vụ án có liên quan.

---

# **Đôi điều về đồ án**

- Mỗi thành viên gặp khó khăn gì?
    - Tập dữ liệu ban đầu bị bias, phải tìm 1 tập dữ khác nên tốn khá nhiều thời gian.
    - Không có đủ thời gian để làm đồ án vì thời gian cận thi cuối kì và phải làm những đồ án khác.
- Mỗi thành viên học được những gì?
    - Học được cách sử dụng git.
    - Biết thêm tình trạng tội phạm ở Mỹ trong những năm gần đây.
    - Biết cách phân bổ thời gian hợp lí.
- Nếu có thêm thời gian, sẽ làm những gì?
    - Xem lại những câu hỏi cũ, có thể là cụ thể hóa những câu hỏi đó để có thể tìm được đáp án gần chính xác nhất.
    - Có thể áp dụng các mô hình học máy khác để tìm ra được mô hình tô nhất, từ đó có thể phục vụ cho các câu hỏi liên quan.

---

# **Các nguồn tham khảo**

https://www.kaggle.com/code/nevil7/homicide-data-identifying-the-serial-killers
https://www.geeksforgeeks.org/pandas-cut-method-in-python/
https://stackoverflow.com/questions/32093829/remove-duplicates-from-dataframe-based-on-two-columns-a-b-keeping-row-with-max
https://sparkbyexamples.com/pandas/pandas-remove-duplicate-columns-from-dataframe/#:~:text=To%20drop%20duplicate%20columns%20from%20pandas%20DataFrame%20use%20df.,data%20regardless%20of%20column%20names